# Cosmos Shell

The Cosmos Shell, is an acronym + visual pun.

    Cosmos: "Cascading Operating SysteM Operating System"
    Shell: "@"
    
## Subclasses APath

The Cosmos Shell python object `CosmosShell` subclasses the `APath` Access Path object.
    
    class CosmosShell(APath):
        ...
    
### Command Execution Environment

The shell uses the Access Path as a context to execute commands.

    cs = CosmosShell(target)
    cs.run(path_to_command, *args, *opts)
    

### Cosmos type system API

The Cosmos type system has commands like `new` which create a file system object from a type template. The `CosmosShell` class provides this API.

    cs.new(new_path, typename, *args, *opts)

### Virtual Filesystem Overlay on Cosmos config directories

If a config directory exists in the underlying filesystem, and its name matches one of those in the `.cosm/etc/cosmdirname` property of the CosmosShell instance, then it's contents will be virtualized to inherit the `.cosm` property context from the parent APath's in the APath chain.

WARNING: The system allows for more than one config directory. The virtual file system will preserve the contents of the directory, plus the blend of inherited directories from the parent paths. If multiple config directories exist, their contents will remain separate until a child APath is created, at which point, they will be blended as expected.

Eg, if `@` and `.cosm` exist, they can each contain separate configurations. They will each inherit the combination of configurations from the parent path and for subdirectories of the current APath, the two will both contribute inheritance.

The configuration inheritance method is implemented in `APath.py`. The virtual directory overlays for file management purposes are implemented here in `CosmosShell`. 


## Target Paths vs. APath Location

In the examples below, a target path may be specified like `/a/b/c.typename`. A CosmosShell operates only on one segment of a path. The path lookup mechanism can use CosmosShell to execute commands in each lookup step, in the context of each path segment, the resulting `APath` object for `/a/b/` would issue a command `new` to create `c.typename`. This command would run in a CosmosShell wrapping the `/a/b/` APath. The pathname value of the APath would be `/a/b/` and the new target directory entry pathname would be `/a/b/c.typename` so the target pathname could be pattern matched with a type-match of `*.typename`. However, it must be noted that the actual execution environment of the CosmosShell is the APath for `/a/b/`, and not `/`.


## Snail Shell Directories

    @/

Anywhere in the file system where there is an `@` directory, the contents of the `@` directory overload the contents of a previous `@` directory further up the hierarchy.

In an `@` aware file system, `@` always resolves as a local directory, even if it doesn't exist in the underlying file system.
    
The snail shell directory may be hidden, but functions the same.

    .@/

## Snail Shell Inheritance

If _multiple_ directories exist, who's names match `@/etc/cosmdirname`, then each directory maintains it's own state, _but_ also inherits the state of the parent `.cosm`. If you write to the directory, the contents will be saved in that directory. If you read the directory, the effects of inheritance apply to those contents, and exclude those other siblings with the `cosmdirname` matches. All those siblings are combined to affect the inheritance of a descendent directory however.

    /@/
        ancestor stuff
    /child/
        @/
            stuff
            ancestor stuff
        .@/
            more stuff
            ancestor stuff
        /grandchild
            @/
                new stuff
                stuff
                more stuff
                ancestor stuff


## Object Oriented Type System

    @/Types/

A type system that uses the python `Jinja` template processor on filenames and file contents:
    
`@/Types` defines classes that may be instantiated under the APath's level of the file system hierarchy.

Example:

    /my/projects/@/Types/project
    
Enables...

    > new /my/project/special.project
    
...to instantiate a directory called `special.project/` which is prepopulated with files from `@/Types/project`

The `new` command accepts arguments that will expand to variable names within the `Types/project` template processed by `Jinja`.

    @/Types/project/
        {{main_file}}.py
        
Where `{{main_flename}}.py` has these contents:

    import {{class}} from {{module}}
    
    x = {{class}}({{ttl}})
    
And where `new` was invoked like this:

    > new /my/project/special.project --json '{"main_file":"boris","class":"Bomb","module":"coat_pocket","ttl":"5000"}'
    
Creates:

    /my/project/
        special.project/
            boris.py
            
Where `boris.py` has these contents:

    import Bomb from coat_pocket
    
    x = Bomb(5000)
    

## Python Implementation

CosmosShell class instance method new(). default behavior uses Jinja to process names and file contents. No code solution. Just create a type template under @/Types and pass the variable substitutions to the new command.

To modify and customize the new object behavior, you need to write a `New.py` file under `@/Types/<yourtype>/@/bin/`. The signature of the new function follows:

    def new(apath, jsonargs)





## Python as a cli

When the `new` command is implemented in Python as CLI executable.

    if __name__ == "__main__":
        fs = OSFS('.')
        snail_shell = SnailShellFS(fs)
        


In [1]:
from APath import APath
from MulticelFS import MulticelFS

class CosmosShell(APath):
    def __init__(self, *a, **k):
        super().__init__(*a, **k)
        
    def lookup(self, addr):
        if addr in self.cosm.listdir('/etc/cosmdirname'):
            _targ1 = super().lookup(addr)
            if self.parent:
                _parent_cosm = self.parent.cosm
            else:
                _parent_cosm = APath._rootcosm
            _cosmstack = [Dcel(_parent_cosm,writeable=False),
                          Dcel(_targ1.target,writeable=True)]
            _overlay = Dcel(_cosmstack, service_class=MulticelFS)
            return APath(_overlay)
        return super().lookup(addr)
    
    def new(self, dirent_name='', typename='', **kwargs):
        return None
    

In [2]:
## Test `__init__()` 

from Dcel import Dcel
from fs.osfs import OSFS

a = CosmosShell(Dcel('fs/fs',service_class=OSFS))

In [3]:
## Test virtual directories: `@` and `.cosm`
# vs `.cosm` property
for each in '/','@','.cosm':
    print(a.listdir(each))
print(a.cosm.listdir())

['tmp_updateMultiValue.txt', 'numbers', 'fruit', '.cosm', 'boats']


KeyError: '@'

In [5]:
## Test `tree()`
a.tree()

|-- .cosm
|   |-- bin
|   |   `-- hello
|   |-- env
|   |   `-- PATH
|   |       |-- bin
|   |       |   |-- bin
|   |       |   |   |-- bin
|   |       |   |   |-- tools
|   |       |   |   `-- types
|   |       |   |-- tools
|   |       |   |   |-- bin
|   |       |   |   |-- tools
|   |       |   |   `-- types
|   |       |   `-- types
|   |       |       |-- bin
|   |       |       |-- tools
|   |       |       `-- types
|   |       |-- tools
|   |       |   |-- bin
|   |       |   |   |-- bin
|   |       |   |   |-- tools
|   |       |   |   `-- types
|   |       |   |-- tools
|   |       |   |   |-- bin
|   |       |   |   |-- tools
|   |       |   |   `-- types
|   |       |   `-- types
|   |       |       |-- bin
|   |       |       |-- tools
|   |       |       `-- types
|   |       `-- types
|   |           |-- bin
|   |           |   |-- bin
|   |           |   |-- tools
|   |           |   `-- types
|   |           |-- tools
|   |           |   |-- bin
|   |           |   |

|   |   |   |   |   |-- .@
|   |   |   |   |   |-- .cosm
|   |   |   |   |   |-- @
|   |   |   |   |   |-- _@
|   |   |   |   |   `-- _cosm
|   |   |   |   |-- _@
|   |   |   |   |   |-- .@
|   |   |   |   |   |-- .cosm
|   |   |   |   |   |-- @
|   |   |   |   |   |-- _@
|   |   |   |   |   `-- _cosm
|   |   |   |   `-- _cosm
|   |   |   |       |-- .@
|   |   |   |       |-- .cosm
|   |   |   |       |-- @
|   |   |   |       |-- _@
|   |   |   |       `-- _cosm
|   |   |   |-- @
|   |   |   |   |-- .@
|   |   |   |   |   |-- .@
|   |   |   |   |   |-- .cosm
|   |   |   |   |   |-- @
|   |   |   |   |   |-- _@
|   |   |   |   |   `-- _cosm
|   |   |   |   |-- .cosm
|   |   |   |   |   |-- .@
|   |   |   |   |   |-- .cosm
|   |   |   |   |   |-- @
|   |   |   |   |   |-- _@
|   |   |   |   |   `-- _cosm
|   |   |   |   |-- @
|   |   |   |   |   |-- .@
|   |   |   |   |   |-- .cosm
|   |   |   |   |   |-- @
|   |   |   |   |   |-- _@
|   |   |   |   |   `-- _cosm
|   |   |   |   |-- _@

In [41]:
## Verify .cosm property from APath inheritance

for ea in '/etc/typematch/filefmt',\
            '/types/fstab',\
            '/types',\
            '/etc/cosmdirname',\
            '.':
    print(a.cosm.listdir(ea))


['.*[/]etc[/]fstab']
['$__start__', 'entry', '#!', 'field']
['cskvp', 'Hello.g4', 'url', 'fstab']
['.@', '_cosm', '.cosm', '@', '_@']
['etc', 'var', 'types', 'WHATISTHIS?', 'hosts']


In [10]:
## Verify contents of cosmos config directories
# in regular filesystem.

from fs import open_fs

r = open_fs('fs/fs')
print(r.listdir('/'))

for each in a.cosm.listdir('/etc/cosmdirname'):
    try:
        print(r.listdir(each))
    except:
        pass

['test', '.cosm', 'fruit', 'tmp_updateMultiValue.txt', 'numbers', 'boats', '@']
['WHATISTHIS?']
['var', 'hosts', 'types', 'etc']


In [25]:
## Test `@` virtual file system

# Expect that the contents of each directory
# matching one of the names in `cosmdirname`
# will be overlayed on the contents
# of the `.cosm` property.

for each in a.cosm.listdir('/etc/cosmdirname'):
    try:
        print(a.listdir(each))
    except:
        pass

['var', 'hosts', 'types', 'etc']
['WHATISTHIS?']


In [29]:
a.readtext('.cosm/etc/fstab')

"# # Cosm / Etc / FSTab\n\n# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.\n\n# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.\n\n# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.\n\n# idea: make filepaths relative to the fstab's location, ie: for ./.cosm/etc/fstab the relative root is ../../../\n\n\n\nfile://raygan@raygan.com.cburn.io/ {cburnuser}/example cburnfs nouser,shortid=RC,idcard=localuser 0 0\nfile://fs2.cburn.io/  {cburnuser}/example/  cburnfs user,shortid=F2,idcard=localuser 0 0\nfile://fs.cburn.io    {cburnuser}/example/  cburnfs user,shortid=F1,idcard=localuser 0 0\nfile://fs4.cburn.io/  {cburnuser}/exam

### WIP `new()` method

In [6]:
a.listdir('/')

['test', '.cosm', 'fruit', 'tmp_updateMultiValue.txt', 'numbers', 'boats', '@']

In [7]:
a.makedir('/test',recreate=True)

SubFS(OSFS('fs/fs'), '/test')

In [8]:
# args
apath = a.opendir('/test')
target = 'new.thing'


In [9]:
apath

SubFS(<APath.APath object at 0x7f4efd63e070>, '/test')

In [14]:
apath.listdir('/')

[]

In [12]:
# won't work if apath is SubFS
apath.cosm.listdir('/')

AttributeError: 'SubFS' object has no attribute 'cosm'

In [15]:
bpath = a.lookup('/test')

In [26]:
print(bpath.cosm.listdir('/types'))
print(bpath.listdir('./'))

['fstab', 'cskvp', 'url']
[]


### Test CosmosShell `init()` method

In [51]:
# Test CosmosShell Init

cosh = CosmosShell(a)

In [52]:
cosh

In [53]:
cosh.new('rclone.conf','*/.config/rclone/rclone.conf')

In [54]:
cosh.new('dbox','*/.config/rclone/rclone.conf/*.dropbox')

In [ ]:
# Test Apath Init Helper for CLI Invocation

CosmosShell.APath_from_cli()